# Using ERA5 together with intake and xarray

In [1]:
import intake
import xarray as xr

In [2]:
catalog = intake.open_esm_datastore("/pool/data/Catalogs/mistral-era5.json")

query = {'level_type':'surface',
         'frequency':'hourly',
         'code':167,
        }

/home/dkrz/k202015/miniconda3/envs/ingest/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3417: DtypeWarning: Columns (5,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
cat =  catalog.search(**query)

In [4]:
cat

,unique
era_id,1
type,1
level_type,1
frequency,1
code,1
validation_date,497
initialization_date,0
step,0
path,497


In [5]:
cat.df.head()

,era_id,type,level_type,frequency,code,validation_date,initialization_date,step,path
0,E5,an,surface,hourly,167,1979-01,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...
1,E5,an,surface,hourly,167,1979-02,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...
2,E5,an,surface,hourly,167,1979-03,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...
3,E5,an,surface,hourly,167,1979-04,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...
4,E5,an,surface,hourly,167,1979-05,NaN,NaN,/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5...


In [7]:
[print(cat.df['path'].iloc[i]) for i in range(0,cat.df['path'].size, cat.df['path'].size//10) ]

/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5sf00_1H_1979-01_167
/mnt/lustre02/work/bk1099/data/sf00_1H/1983/E5sf00_1H_1983-02_167
/mnt/lustre02/work/bk1099/data/sf00_1H/1987/E5sf00_1H_1987-03_167
/mnt/lustre02/work/bk1099/data/sf00_1H/1991/E5sf00_1H_1991-04_167
/mnt/lustre02/work/bk1099/data/sf00_1H/1995/E5sf00_1H_1995-05_167
/mnt/lustre02/work/bk1099/data/sf00_1H/1999/E5sf00_1H_1999-06_167
/mnt/lustre02/work/bk1099/data/sf00_1H/2003/E5sf00_1H_2003-07_167
/mnt/lustre02/work/bk1099/data/sf00_1H/2007/E5sf00_1H_2007-08_167
/mnt/lustre02/work/bk1099/data/sf00_1H/2011/E5sf00_1H_2011-09_167
/mnt/lustre02/work/bk1099/data/sf00_1H/2015/E5sf00_1H_2015-10_167
/mnt/lustre02/work/bk1099/data/sf00_1H/2019/E5sf00_1H_2019-11_167


[None, None, None, None, None, None, None, None, None, None, None]

## open an individual grib file 

- optional '.idx' files are generated,when opening files via cfgrib, set the backend_kwargs option to skip warnings

In [8]:
path1 = cat.df['path'].iloc[1]
path1

'/mnt/lustre02/work/bk1099/data/sf00_1H/1979/E5sf00_1H_1979-02_167'

In [9]:
with xr.open_dataset(path1,engine='cfgrib',backend_kwargs= {'indexpath':''}) as fobj:
    print(fobj.variables)

Frozen({'number': <xarray.Variable ()>
array(0)
Attributes:
    long_name:      ensemble member numerical id
    units:          1
    standard_name:  realization, 'time': <xarray.IndexVariable 'time' (time: 672)>
array(['1979-02-01T00:00:00.000000000', '1979-02-01T01:00:00.000000000',
       '1979-02-01T02:00:00.000000000', ..., '1979-02-28T21:00:00.000000000',
       '1979-02-28T22:00:00.000000000', '1979-02-28T23:00:00.000000000'],
      dtype='datetime64[ns]')
Attributes:
    long_name:      initial time of forecast
    standard_name:  forecast_reference_time, 'step': <xarray.Variable ()>
array(0, dtype='timedelta64[ns]')
Attributes:
    long_name:      time since forecast_reference_time
    standard_name:  forecast_period, 'surface': <xarray.Variable ()>
array(0.)
Attributes:
    long_name:  original GRIB coordinate for key: level(surface)
    units:      1, 'latitude': <xarray.Variable (values: 542080)>
[542080 values with dtype=float64]
Attributes:
    units:          degrees_no

In [11]:
wds = xr.load_dataset(path1,engine='cfgrib',backend_kwargs= {'indexpath':''})
wds

<xarray.Dataset>
Dimensions:     (time: 672, values: 542080)
Coordinates:
    number      int64 0
  * time        (time) datetime64[ns] 1979-02-01 ... 1979-02-28T23:00:00
    step        timedelta64[ns] 00:00:00
    surface     float64 0.0
    latitude    (values) float64 89.78 89.78 89.78 ... -89.78 -89.78 -89.78
    longitude   (values) float64 0.0 20.0 40.0 60.0 ... 280.0 300.0 320.0 340.0
    valid_time  (time) datetime64[ns] 1979-02-01 ... 1979-02-28T23:00:00
Dimensions without coordinates: values
Data variables:
    t2m         (time, values) float32 241.7 241.5 241.3 ... 225.3 225.4 225.5
Attributes:
    GRIB_edition:            1
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2021-04-10T12:56 GRIB to CDM+CF via cfgrib-0.9.9...

## load a large set of files directly from the catalog into a lazy xarray

In [12]:
dts = cat.to_dataset_dict(cdf_kwargs={"chunks": {"time": 36},"engine":'cfgrib',"backend_kwargs": {'indexpath':''}}) 
dts


--> The keys in the returned dictionary of datasets are constructed as follows:
	'era_id.type.level_type.frequency.code.validation_date.initialization_date.step.path'


{'E5.an.surface.hourly.167.1992-03.nan.nan./mnt/lustre02/work/bk1099/data/sf00_1H/1992/E5sf00_1H_1992-03_167': <xarray.Dataset>
 Dimensions:     (time: 744, values: 542080)
 Coordinates:
     number      int64 ...
   * time        (time) datetime64[ns] 1992-03-01 ... 1992-03-31T23:00:00
     step        timedelta64[ns] ...
     surface     float64 ...
     latitude    (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
     longitude   (values) float64 dask.array<chunksize=(542080,), meta=np.ndarray>
     valid_time  (time) datetime64[ns] dask.array<chunksize=(36,), meta=np.ndarray>
 Dimensions without coordinates: values
 Data variables:
     t2m         (time, values) float32 dask.array<chunksize=(36, 542080), meta=np.ndarray>
 Attributes:
     GRIB_edition:            1
     GRIB_centre:             ecmf
     GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
     GRIB_subCentre:          0
     Conventions:             CF-1.7
     institution: